In [34]:
from serial import Serial
from pyubx2 import UBXReader, NMEA_PROTOCOL, UBX_PROTOCOL
from pyubx2 import UBXMessage, SET_LAYER_RAM, TXN_NONE
from pyubx2 import POLL_LAYER_RAM

In [35]:
usb_port = '/dev/ttyACM0'
baud_rate = 38400
set_layers = SET_LAYER_RAM
poll_layers = POLL_LAYER_RAM
transaction = TXN_NONE

In [49]:
def ubx_set(cfgData, layers=set_layers, transaction=transaction, verbose=False):
    msg = UBXMessage.config_set(layers, transaction, cfgData)
    if verbose:
        print(msg)
    else:
        with Serial(usb_port, baud_rate, timeout=3) as stream:
            stream.write(msg.serialize())

def ubx_poll(keys, layers=poll_layers,position=0, verbose=False):
    msg = UBXMessage.config_poll(layers, position, keys)
    if verbose:
        print(msg)
    else:
        with Serial(usb_port, baud_rate, timeout=3) as stream:
            stream.write(msg.serialize())
            ubr = UBXReader(stream, protfilter=NMEA_PROTOCOL | UBX_PROTOCOL)
            raw_data, parsed_data = ubr.read()
            return parsed_data
    

In [33]:
from serial import Serial
from pyubx2 import UBXReader, NMEA_PROTOCOL, UBX_PROTOCOL
with Serial(usb_port, baud_rate, timeout=3) as stream:
  ubr = UBXReader(stream, protfilter=NMEA_PROTOCOL | UBX_PROTOCOL)
  raw_data, parsed_data = ubr.read()
  if parsed_data is not None:
    print(parsed_data)

<NMEA(GNRMC, time=00:08:10, status=A, lat=37.8729406667, NS=N, lon=-122.2572496667, EW=W, spd=0.112, cog=, date=2024-06-26, mv=, mvEW=, posMode=A, navStatus=V)>


In [48]:
# USB input: NMEA, UBX 
keys = [("CFG_USBINPROT_NMEA", 1), ("CFG_USBINPROT_UBX", 1)]
ubx_set(keys, verbose=True)
# USB output: UBX
keys = [("CFG_USBOUTPROT_NMEA", 1), ("CFG_USBOUTPROT_UBX", 1)]
ubx_set(keys, verbose=False)



<UBX(CFG-VALSET, version=0, ram=1, bbr=0, flash=0, action=0, reserved0=0, CFG_USBINPROT_NMEA=1, CFG_USBINPROT_UBX=1)>


In [44]:
keys=["CFG_USBOUTPROT_NMEA"]
ubx_poll(keys, verbose=True)

<UBX(CFG-VALGET, version=0, layer=0, position=0, keys_01=276299778)>


In [47]:
keys=["CFG_USBOUTPROT_NMEA"]
ubx_poll(keys)

<UBX(CFG-VALGET, version=1, layer=0, position=0, CFG_USBOUTPROT_NMEA=0)>


In [52]:
# set the GPS receiver work in survey-in mode
# survey-in minimum duration is 1200s
# survey-in position accuracy limit 0.1m
keys = [("CFG_TMODE_MODE", 1), ("CFG_TMODE_SVIN_MIN_DUR", 1200), ("CFG_TMODE_SVIN_ACC_LIMIT", 1000)]
ubx_set(keys, verbose=False)